In [1]:
import numpy as np
import torch
from torchsummary import summary
import os

import sys
sys.path.insert(1, 'src')

from network import BF_CNN
from universal_inverse_solve import universal_inverse_solver

## Load Denoiser

In [2]:
# Load the denoiser architecture
denoiser = BF_CNN()

# Use GPU if available
if torch.cuda.is_available():
    denoiser = denoiser.cuda()

# Load the learned parameters
denoiser_path = os.path.join('denoisers','mnist.pt')

if torch.cuda.is_available():
    learned_params =torch.load(denoiser_path)
else:
    learned_params =torch.load(denoiser_path, map_location='cpu')

denoiser.load_state_dict(learned_params)

# Show summary of the denoiser
summary(denoiser, (1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
├─ModuleList: 1                          []                        --
|    └─Conv2d: 2-1                       [-1, 64, 28, 28]          576
|    └─Conv2d: 2-2                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-3                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-4                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-5                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-6                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-7                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-8                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-9                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-10                      [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-11                      [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-12                      [-1

Layer (type:depth-idx)                   Output Shape              Param #
├─ModuleList: 1                          []                        --
|    └─Conv2d: 2-1                       [-1, 64, 28, 28]          576
|    └─Conv2d: 2-2                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-3                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-4                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-5                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-6                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-7                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-8                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-9                       [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-10                      [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-11                      [-1, 64, 28, 28]          36,864
|    └─Conv2d: 2-12                      [-1

# Synthesis

In [4]:
# def universal_inverse_solver(M, M_T, x_c, denoiser, sigma_0=1, sigma_L=0.01, h_0=0.01, beta=0.01):


# For synthesis, both M and M_T are zero like or shape sample image
# Images are mnist, so 28x28
M = np.zeros_like(28)
M_T = np.zeros_like(28)

# Input image is 28x28 of zeros and noise will be added to it
# This will be synthesized by the denoiser

patch_size = (1, 28, 28) # 1 channel, 28x28
x_c = torch.zeros(patch_size) 


synth = universal_inverse_solver(M, M_T, x_c, denoiser)

TypeError: matmul(): argument 'input' (position 1) must be Tensor, not numpy.ndarray